# Data Prep Kit Demo 1 - Ray Version

This notebook will introduce DPK and showcase some of it's capabilities.

Here is the workflow

![](https://raw.githubusercontent.com/sujee/data-prep-kit-examples/main/media/data-prep-kit-3-workflow.png)

References
- [Data prep kit](https://ibm.github.io/data-prep-kit)  is an open source framework that helps with data wrangling.
- [github repo](https://github.com/IBM/data-prep-kit)


## How to run this notebook

Two options:

- **Option 1 - Google Colab:** easiest option.  no setup required.  Click this link to open this on google colab.  [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sujee/data-prep-kit-examples/blob/main/dpk-intro/process_pdfs_1_ray.ipynb)
- **Option 2 - Local python dev environment:**  Setup using this guide: [setting up python dev environment](../setup-python-dev-env.md)

The notebook will work as in both environments

## Step-1: Inspect the Data

We will use simple PDFs about Solar system.  The files are [here](https://github.com/sujee/data-prep-kit-examples/tree/main/data/solar-system)

- [earth.pdf](https://github.com/sujee/data-prep-kit-examples/blob/main/data/solar-system/earth.pdf)
- [mars.pdf](https://github.com/sujee/data-prep-kit-examples/blob/main/data/solar-system/mars.pdf)

### (Optional) How to create PDFs?

If you like to play around with various inputs files, follow these steps to re-generate PDFs.

**Option 1 (Easiest): Use a word editor or google docs editor**

Write your content and export as PDF


**Option 2: markdown -> pdf**

First edit the markdown files using any text editor.

Then use [pandoc](https://pandoc.org/) to convert them to pdfs.

```bash
pandoc  earth.md  -o earth.pdf
pandoc  mars.md  -o mars.pdf
```


## Step-2: Figure out Runtime Environment

### 2.1 - Determine runtime

Determine if we are running on Google colab or local python environment

In [1]:
import os

if os.getenv("COLAB_RELEASE_TAG"):
   print("Running in Colab")
   RUNNING_IN_COLAB = True
else:
   print("NOT in Colab")
   RUNNING_IN_COLAB = False

NOT in Colab


### 2.2 -Download Data if running on Google Colab

In [2]:
if RUNNING_IN_COLAB:
    !mkdir -p 'input'
    !wget -O 'input/earth.pdf'  'https://raw.githubusercontent.com/sujee/data-prep-kit-examples/main/data/solar-system/earth.pdf'
    !wget -O 'input/mars.pdf'  'https://raw.githubusercontent.com/sujee/data-prep-kit-examples/main/data/solar-system/mars.pdf'
    !wget -O 'utils.py'  'https://raw.githubusercontent.com/sujee/data-prep-kit-examples/main/utils.py'

### 2.3 - Install dependencies if running on Google Colab

In [3]:
if RUNNING_IN_COLAB:
    ! pip install  --default-timeout=100  \
        data-prep-toolkit==0.2.2 \
        data-prep-toolkit-transforms[all,ray]==0.2.2 \
        humanfriendly

### 2.4 - Restart Runtime

After installing dependencies, be sure <font color="red">restart runtime</font>, so libraries will be loaded

You do this by going to **`Runtime --> Restart Session`**

Then you can continue to the next step (no need to re-run the notebook)

## Step-2: Configuration

### 2.1 - Basic Config

In [4]:
import os

if os.getenv("COLAB_RELEASE_TAG"):
   print("Running in Colab")
   RUNNING_IN_COLAB = True
else:
   print("NOT in Colab")
   RUNNING_IN_COLAB = False

NOT in Colab


In [5]:
import os

## Configuration
class MyConfig:
    pass

MY_CONFIG = MyConfig ()

if RUNNING_IN_COLAB:
  MY_CONFIG.INPUT_DATA_DIR = 'input'
else:
  MY_CONFIG.INPUT_DATA_DIR = os.path.join (os.path.abspath (''), '..', 'data', 'solar-system')
MY_CONFIG.OUTPUT_FOLDER = "output"
MY_CONFIG.OUTPUT_FOLDER_FINAL = os.path.join(MY_CONFIG.OUTPUT_FOLDER , "output_final")

## Embedding model
MY_CONFIG.EMBEDDING_MODEL = 'sentence-transformers/all-MiniLM-L6-v2'

## RAY CONFIGURATION
### For local runs, we can use more parallelism
### For google colab, be conservative

if RUNNING_IN_COLAB:
  MY_CONFIG.RAY_RUNTIME_WORKERS = 2
  MY_CONFIG.RAY_NUM_CPUS =  0.3
  MY_CONFIG.RAY_MEMORY_GB = 2  # GB
else:  # local run
  num_cpus_available =  os.cpu_count()
  # print (num_cpus_available)
  MY_CONFIG.RAY_NUM_CPUS =  1
  MY_CONFIG.RAY_MEMORY_GB = 2  # GB
  # MY_CONFIG.RAY_RUNTIME_WORKERS = num_cpus_available // 3
  MY_CONFIG.RAY_RUNTIME_WORKERS = 2

print ('MY_CONFIG.RAY_RUNTIME_WORKERS:', MY_CONFIG.RAY_RUNTIME_WORKERS)
print ('MY_CONFIG.RAY_NUM_CPUS:', MY_CONFIG.RAY_NUM_CPUS)
print ('MY_CONFIG.RAY_MEMORY_GB:', MY_CONFIG.RAY_MEMORY_GB)


MY_CONFIG.RAY_RUNTIME_WORKERS: 2
MY_CONFIG.RAY_NUM_CPUS: 1
MY_CONFIG.RAY_MEMORY_GB: 2


In [6]:
## Add parent dir to path
import os,sys

this_dir = os.path.abspath('')
parent_dir = os.path.dirname(this_dir)
sys.path.append (os.path.abspath (parent_dir))

### 2.2 - Setup input/outpur directories

In [7]:
import os, sys
import shutil

if not os.path.exists(MY_CONFIG.INPUT_DATA_DIR ):
    raise Exception (f"❌ Input folder MY_CONFIG.INPUT_DATA_DIR = '{MY_CONFIG.INPUT_DATA_DIR}' not found")

output_parquet_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '01_parquet_out')
output_chunk_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '02_chunk_out')
output_docid_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '03_docid_out')
output_exact_dedupe_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '04_exact_dedupe_out')
output_fuzzy_dedupe_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '05_fuzzy_dedupe_out')
output_embeddings_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '06_embeddings_out')

## clear output folder
shutil.rmtree(MY_CONFIG.OUTPUT_FOLDER, ignore_errors=True)
shutil.os.makedirs(MY_CONFIG.OUTPUT_FOLDER, exist_ok=True)

print ("✅ Cleared output directory")

✅ Cleared output directory


## Step-3: pdf2parquet -  Convert data from PDF to Parquet

This step is reading the input folder containing all PDF files and ingest them in a parquet table using the [Docling package](https://github.com/DS4SD/docling).
The documents are converted into a JSON format which allows to easily chunk it in the later steps.



### 3.1 - Set Input/output Folder

In [8]:
STAGE = 1

input_folder = MY_CONFIG.INPUT_DATA_DIR
output_folder =  output_parquet_dir

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-1: Processing input='/home/sujee/my-stuff/projects/ai-alliance/data-prep-kit-examples/dpk-intro/../data/solar-system' --> output='output/01_parquet_out'


### 3.2 - Execute

In [9]:
%%time

import ast
import os
import sys

from pdf2parquet_transform import (
    pdf2parquet_contents_type_cli_param,
    pdf2parquet_contents_types,
)
from data_processing_ray.runtime.ray import RayTransformLauncher
from pdf2parquet_transform_python import Pdf2ParquetPythonTransformConfiguration
from pdf2parquet_transform_ray import Pdf2ParquetRayTransformConfiguration

from data_processing.utils import GB, ParamsUtils


# create parameters
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS, "memory": MY_CONFIG.RAY_MEMORY_GB * GB}
ingest_config = {
    pdf2parquet_contents_type_cli_param: pdf2parquet_contents_types.JSON,
}

params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    "data_files_to_use": ast.literal_eval("['.pdf']"),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    # "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    "runtime_num_workers": 1,  # setting it to 1 for this instance
}


sys.argv = ParamsUtils.dict_to_req(d=(params | ingest_config))
# create launcher
launcher = RayTransformLauncher(Pdf2ParquetRayTransformConfiguration())
# launcher = PythonTransformLauncher(Pdf2ParquetPythonTransformConfiguration())
# launch
return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")


23:41:18 INFO - pdf2parquet parameters are : {'batch_size': -1, 'artifacts_path': None, 'contents_type': <pdf2parquet_contents_types.JSON: 'application/json'>, 'do_table_structure': True, 'do_ocr': True, 'ocr_engine': <pdf2parquet_ocr_engine.EASYOCR: 'easyocr'>, 'bitmap_area_threshold': 0.05, 'pdf_backend': <pdf2parquet_pdf_backend.DLPARSE_V2: 'dlparse_v2'>, 'double_precision': 8}
23:41:18 INFO - pipeline id pipeline_id
23:41:18 INFO - code location None
23:41:18 INFO - number of workers 1 worker options {'num_cpus': 1, 'memory': 2147483648, 'max_restarts': -1}
23:41:18 INFO - actor creation delay 0
23:41:18 INFO - job details {'job category': 'preprocessing', 'job name': 'pdf2parquet', 'job type': 'ray', 'job id': 'job_id'}
23:41:18 INFO - data factory data_ is using local data access: input_folder - /home/sujee/my-stuff/projects/ai-alliance/data-prep-kit-examples/dpk-intro/../data/solar-system output_folder - output/01_parquet_out
23:41:18 INFO - data factory data_ max_files -1, n_sa

✅ Stage:1 completed successfully
CPU times: user 4.61 s, sys: 1.26 s, total: 5.88 s
Wall time: 29.6 s


### 3.3 - Inspect Generated output

Here we should see one entry per input file processed.

In [10]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Output dimensions (rows x columns)= ", output_df.shape)

output_df.head(5)

## To display certain columns
#parquet_df[['column1', 'column2', 'column3']].head(5)

Output dimensions (rows x columns)=  (2, 13)


,filename,contents,num_pages,num_tables,num_doc_elements,document_id,document_hash,ext,hash,size,date_acquired,pdf_convert_time,source_filename
0,mars.pdf,"{""schema_name"":""DoclingDocument"",""version"":""1....",1,0,11,1554b6dd-8b79-4d75-9005-d39dfdb1439e,10359780639229817778,pdf,28aa3d03f33e8554459437f647fafca657b5360d0f7332...,5220,2024-12-03T23:41:33.764282,1.671919,mars.pdf
1,earth.pdf,"{""schema_name"":""DoclingDocument"",""version"":""1....",1,0,11,7cb164c0-cadf-4b65-a525-3c2d1a553128,17915699055171962696,pdf,04e935746bc6dfb41912dd108bb10dde21c60b0da3f8fa...,4992,2024-12-03T23:41:32.088316,1.833233,earth.pdf



### 3.4 - Understand the output

Here are some interesting attributes to note:

- **filename** : original filename
- **contents** : text
- **document_id**: unique id (UUID) assignd to this document
- **hash** : hash of document
- **pdf_convert_time** : time to convert this pdf in seconds

Let's inspect the **contents** column.  See how the text is being divided up!

In [11]:
import pprint
import json

pprint.pprint (json.loads(output_df.iloc[0, ]['contents']))
# json.loads(output_df.iloc[0, ]['contents'])

{'body': {'children': [{'$ref': '#/texts/0'},
                       {'$ref': '#/texts/1'},
                       {'$ref': '#/texts/2'},
                       {'$ref': '#/texts/3'},
                       {'$ref': '#/texts/4'},
                       {'$ref': '#/texts/5'},
                       {'$ref': '#/texts/6'},
                       {'$ref': '#/groups/0'},
                       {'$ref': '#/texts/10'}],
          'label': 'unspecified',
          'name': '_root_',
          'self_ref': '#/body'},
 'furniture': {'children': [],
               'label': 'unspecified',
               'name': '_root_',
               'self_ref': '#/furniture'},
 'groups': [{'children': [{'$ref': '#/texts/7'},
                          {'$ref': '#/texts/8'},
                          {'$ref': '#/texts/9'}],
             'label': 'list',
             'name': 'list',
             'parent': {'$ref': '#/body'},
             'self_ref': '#/groups/0'}],
 'key_value_items': [],
 'name': 'mars',
 'origin':

In [12]:
pprint.pprint (json.loads(output_df.iloc[1, ]['contents']))

{'body': {'children': [{'$ref': '#/texts/0'},
                       {'$ref': '#/texts/1'},
                       {'$ref': '#/texts/2'},
                       {'$ref': '#/texts/3'},
                       {'$ref': '#/texts/4'},
                       {'$ref': '#/texts/5'},
                       {'$ref': '#/texts/6'},
                       {'$ref': '#/groups/0'},
                       {'$ref': '#/texts/10'}],
          'label': 'unspecified',
          'name': '_root_',
          'self_ref': '#/body'},
 'furniture': {'children': [],
               'label': 'unspecified',
               'name': '_root_',
               'self_ref': '#/furniture'},
 'groups': [{'children': [{'$ref': '#/texts/7'},
                          {'$ref': '#/texts/8'},
                          {'$ref': '#/texts/9'}],
             'label': 'list',
             'name': 'list',
             'parent': {'$ref': '#/body'},
             'self_ref': '#/groups/0'}],
 'key_value_items': [],
 'name': 'earth',
 'origin'

##  Step-4: Doc chunks

In the previous step, we have extracted text from oru PDFs.  But we have the content of entire file as 'one row' in our parquet output.

In this step, we are going to split the documents in chunks, according to their layout segmentation.

This transform uses [Quackling](https://github.com/DS4SD/quackling) `HierarchicalChunker`
to chunk according to the document layout segmentation, i.e. respecting the original document components as paragraphs, tables, enumerations, etc.
It relies on documents converted with the Docling library in the [pdf2parquet transform](https://github.com/IBM/data-prep-kit/blob/dev/transforms/language/pdf2parquet/python/README.md) using the option `contents_type: "application/json"`,
which provides the required JSON structure.

### 4.1 - Set Input/output Folder

In [13]:
STAGE = 2

input_folder = output_parquet_dir # previous output folder is the input folder for the current stage
output_folder =  output_chunk_dir

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-2: Processing input='output/01_parquet_out' --> output='output/02_chunk_out'


### 4.2 - Execute

In [14]:
%%time

from data_processing_ray.runtime.ray import RayTransformLauncher
from doc_chunk_transform_ray import DocChunkRayTransformConfiguration


# Prepare the commandline params
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # doc_chunk arguments
    # ...
}

# Pass the commandline params
sys.argv = ParamsUtils.dict_to_req(d=params)

# create launcher
launcher = RayTransformLauncher(DocChunkRayTransformConfiguration())
# launch
return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

23:41:46 INFO - doc_chunk parameters are : {'chunking_type': <chunking_types.DL_JSON: 'dl_json'>, 'content_column_name': 'contents', 'doc_id_column_name': 'document_id', 'output_chunk_column_name': 'contents', 'output_source_doc_id_column_name': 'source_document_id', 'output_jsonpath_column_name': 'doc_jsonpath', 'output_pageno_column_name': 'page_number', 'output_bbox_column_name': 'bbox', 'chunk_size_tokens': 128, 'chunk_overlap_tokens': 30, 'dl_min_chunk_len': None}
23:41:46 INFO - pipeline id pipeline_id
23:41:46 INFO - code location None
23:41:46 INFO - number of workers 2 worker options {'num_cpus': 1, 'max_restarts': -1}
23:41:46 INFO - actor creation delay 0
23:41:46 INFO - job details {'job category': 'preprocessing', 'job name': 'doc_chunk', 'job type': 'ray', 'job id': 'job_id'}
23:41:46 INFO - data factory data_ is using local data access: input_folder - output/01_parquet_out output_folder - output/02_chunk_out
23:41:46 INFO - data factory data_ max_files -1, n_sample -1
23

✅ Stage:2 completed successfully
CPU times: user 945 ms, sys: 268 ms, total: 1.21 s
Wall time: 17.8 s


### 4.3 - Inspect Generated output

We would see documents are split into many chunks

In [15]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print (f"Files processed : {input_df.shape[0]:,}")
print (f"Chunks created : {output_df.shape[0]:,}")

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.head(10)

Files processed : 2
Chunks created : 11
Input data dimensions (rows x columns)=  (2, 13)
Output data dimensions (rows x columns)=  (11, 17)


,filename,num_pages,num_tables,num_doc_elements,document_hash,ext,hash,size,date_acquired,pdf_convert_time,source_filename,source_document_id,contents,doc_jsonpath,page_number,bbox,document_id
0,mars.pdf,1,0,11,10359780639229817778,pdf,28aa3d03f33e8554459437f647fafca657b5360d0f7332...,5220,2024-12-03T23:41:33.764282,1.671919,mars.pdf,1554b6dd-8b79-4d75-9005-d39dfdb1439e,Our solar system is a vast and fascinating exp...,#/texts/2,1,"[132.68968201, 588.23297119, 479.40899658, 626...",4d2e06dbe1d7a07f3722947cb6e7f9e2487d4a536c73dc...
1,mars.pdf,1,0,11,10359780639229817778,pdf,28aa3d03f33e8554459437f647fafca657b5360d0f7332...,5220,2024-12-03T23:41:33.764282,1.671919,mars.pdf,1554b6dd-8b79-4d75-9005-d39dfdb1439e,For more details about the Solar system see Ch...,#/texts/3,1,"[133.21472168, 570.30102539, 374.99798584, 584...",39a860697c7d1af540aa3134b499d476e713da79691077...
2,mars.pdf,1,0,11,10359780639229817778,pdf,28aa3d03f33e8554459437f647fafca657b5360d0f7332...,5220,2024-12-03T23:41:33.764282,1.671919,mars.pdf,1554b6dd-8b79-4d75-9005-d39dfdb1439e,"Mars, the fourth planet from the Sun, is a col...",#/texts/5,1,"[132.89646912, 500.11300659, 477.48300171, 538...",d3ea11dc437075f40b7de47fd2b8a1d334cbb13a20eb22...
3,mars.pdf,1,0,11,10359780639229817778,pdf,28aa3d03f33e8554459437f647fafca657b5360d0f7332...,5220,2024-12-03T23:41:33.764282,1.671919,mars.pdf,1554b6dd-8b79-4d75-9005-d39dfdb1439e,· Distance from the Sun: Average of 228 millio...,#/texts/7,1,"[145.94500732, 452.29199219, 477.48199463, 478...",78223c017bd6ad6c4bd9dd9b5b5d5974c875c559a0be9f...
4,mars.pdf,1,0,11,10359780639229817778,pdf,28aa3d03f33e8554459437f647fafca657b5360d0f7332...,5220,2024-12-03T23:41:33.764282,1.671919,mars.pdf,1554b6dd-8b79-4d75-9005-d39dfdb1439e,1,#/texts/10,1,"[303.02359009, 86.47599792, 308.11401367, 100....",6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...
5,earth.pdf,1,0,11,17915699055171962696,pdf,04e935746bc6dfb41912dd108bb10dde21c60b0da3f8fa...,4992,2024-12-03T23:41:32.088316,1.833233,earth.pdf,7cb164c0-cadf-4b65-a525-3c2d1a553128,Our solar system is a vast and fascinating exp...,#/texts/2,1,"[132.6648407, 588.23297119, 479.40899658, 626....",4d2e06dbe1d7a07f3722947cb6e7f9e2487d4a536c73dc...
6,earth.pdf,1,0,11,17915699055171962696,pdf,04e935746bc6dfb41912dd108bb10dde21c60b0da3f8fa...,4992,2024-12-03T23:41:32.088316,1.833233,earth.pdf,7cb164c0-cadf-4b65-a525-3c2d1a553128,For more details about our Solar system see Ch...,#/texts/3,1,"[133.18649292, 570.30102539, 375.57901001, 584...",6b1799aadb8131a788033660ec73788c9f4165eda5e339...
7,earth.pdf,1,0,11,17915699055171962696,pdf,04e935746bc6dfb41912dd108bb10dde21c60b0da3f8fa...,4992,2024-12-03T23:41:32.088316,1.833233,earth.pdf,7cb164c0-cadf-4b65-a525-3c2d1a553128,Earth is the third planet from the Sun. It's o...,#/texts/5,1,"[133.06599426, 512.06799316, 477.86444092, 538...",0408c7d1d9f634bdcc5d56eb76c4ab4578d8347fe94d9b...
8,earth.pdf,1,0,11,17915699055171962696,pdf,04e935746bc6dfb41912dd108bb10dde21c60b0da3f8fa...,4992,2024-12-03T23:41:32.088316,1.833233,earth.pdf,7cb164c0-cadf-4b65-a525-3c2d1a553128,Basic facts about Earth:,#/texts/6,1,"[133.3850708, 494.13500977, 240.17199707, 508....",117c26c32f0a3ba6bfb7ccafab09213e415296ae97150d...
9,earth.pdf,1,0,11,17915699055171962696,pdf,04e935746bc6dfb41912dd108bb10dde21c60b0da3f8fa...,4992,2024-12-03T23:41:32.088316,1.833233,earth.pdf,7cb164c0-cadf-4b65-a525-3c2d1a553128,· Distance from the Sun: Average of 149.6 mill...,#/texts/7,1,"[145.94500732, 464.24700928, 477.48001099, 490...",83956bf14ac5123da4cb32147974bc6d13b0a8e9368f47...


### 4.4 - Understanding the Output

Here we see 2 PDF files are split into 6 chunks.  Basically we see the documents are being split along 'natural boundaris' - paragraphs and bullet points

See how **document_id** is carried throughout.  This helps us identify original documents.

Also note **contents** is now plain text (not JSON as before)

In [16]:
output_df[['filename', 'contents']]

,filename,contents
0,mars.pdf,Our solar system is a vast and fascinating exp...
1,mars.pdf,For more details about the Solar system see Ch...
2,mars.pdf,"Mars, the fourth planet from the Sun, is a col..."
3,mars.pdf,· Distance from the Sun: Average of 228 millio...
4,mars.pdf,1
5,earth.pdf,Our solar system is a vast and fascinating exp...
6,earth.pdf,For more details about our Solar system see Ch...
7,earth.pdf,Earth is the third planet from the Sun. It's o...
8,earth.pdf,Basic facts about Earth:
9,earth.pdf,· Distance from the Sun: Average of 149.6 mill...


In [17]:
for f in output_df['filename'].unique():
    print ('==========' , f, '===========')
    chunks = output_df[output_df['filename'] == f]['contents']
    for idx , chunk in enumerate(chunks):
        print (f'-------Chunk {idx}------\n{chunk}\n-------')

========== mars.pdf ===========
-------Chunk 0------
Our solar system is a vast and fascinating expanse, comprising eight planets, ﬁve dwarf planets, numerous moons, asteroids, comets, and other celestial bodies. At its center lies the star we call the Sun.
-------
-------Chunk 1------
For more details about the Solar system see Chapter 1.
-------
-------Chunk 2------
Mars, the fourth planet from the Sun, is a cold, desert world with a thin atmosphere composed primarily of carbon dioxide. Its reddish hue comes from iron oxide, or rust, prevalent on its surface.
-------
-------Chunk 3------
· Distance from the Sun: Average of 228 million kilometers (142 million miles)
· Moons: Two small moons, Phobos and Deimos.
· Rotation Period: 24.6 hours (one Martian day - called a "sol")
-------
-------Chunk 4------
1
-------
========== earth.pdf ===========
-------Chunk 0------
Our solar system is a vast and fascinating expanse, comprising eight planets, ﬁve dwarf planets, numerous moons, asteroid

## Step-5:  DOC ID generation

This transform annotates documents with document "ids". It supports the following transformations of the original data:

 - Adding document hash: this enables the addition of a document hash-based id to the data. The hash is calculated with `hashlib.sha256(doc.encode("utf-8")).hexdigest()`. To enable this annotation, set **hash_column** to the name of the column, where you want to store it.
 - Adding integer document id: this allows the addition of an integer document id to the data that is unique across all rows in all tables provided to the transform() method. To enable this annotation, set **int_id_column** to the name of the column, where you want to store it.

**This is a pre-requisite for fuzzy dedup** in the pipeline.

### 5.1 - Set Input/output Folder

In [18]:

# Input for this stage is the output of exact dedeup component
# output of this component makes it possible for fdedup component to run on data.

STAGE  = 3

input_folder = output_chunk_dir
output_folder =  output_docid_dir

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-3: Processing input='output/02_chunk_out' --> output='output/03_docid_out'


### 5.2 - Execute

In [19]:
%%time

from data_processing_ray.runtime.ray import RayTransformLauncher
from doc_id_transform_ray import DocIDRayTransformRuntimeConfiguration

local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # doc id configuration
    "doc_id_doc_column": "contents",
    "doc_id_hash_column": "chunk_hash",
    "doc_id_int_column": "chunk_id",
}
sys.argv = ParamsUtils.dict_to_req(d=params)

# launch

launcher = RayTransformLauncher(DocIDRayTransformRuntimeConfiguration())

return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

23:42:03 INFO - Doc id parameters are : {'doc_column': 'contents', 'hash_column': 'chunk_hash', 'int_column': 'chunk_id', 'start_id': 0}
23:42:03 INFO - pipeline id pipeline_id
23:42:03 INFO - code location None
23:42:03 INFO - number of workers 2 worker options {'num_cpus': 1, 'max_restarts': -1}
23:42:03 INFO - actor creation delay 0
23:42:03 INFO - job details {'job category': 'preprocessing', 'job name': 'doc_id', 'job type': 'ray', 'job id': 'job_id'}
23:42:03 INFO - data factory data_ is using local data access: input_folder - output/02_chunk_out output_folder - output/03_docid_out
23:42:03 INFO - data factory data_ max_files -1, n_sample -1
23:42:03 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
23:42:03 INFO - Running locally
2024-12-03 23:42:04,173	INFO worker.py:1777 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
(orchestrate pid=628

✅ Stage:3 completed successfully
CPU times: user 123 ms, sys: 127 ms, total: 250 ms
Wall time: 14.7 s


### 5.3 - Inspect Generated output

You will notice we have two extra columns

- **hash_column**
- **int_id_column**

But still the same number or rows as before

In [20]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.head(10)

Input data dimensions (rows x columns)=  (11, 17)
Output data dimensions (rows x columns)=  (11, 19)


,filename,num_pages,num_tables,num_doc_elements,document_hash,ext,hash,size,date_acquired,pdf_convert_time,source_filename,source_document_id,contents,doc_jsonpath,page_number,bbox,document_id,chunk_hash,chunk_id
0,mars.pdf,1,0,11,10359780639229817778,pdf,28aa3d03f33e8554459437f647fafca657b5360d0f7332...,5220,2024-12-03T23:41:33.764282,1.671919,mars.pdf,1554b6dd-8b79-4d75-9005-d39dfdb1439e,Our solar system is a vast and fascinating exp...,#/texts/2,1,"[132.68968201, 588.23297119, 479.40899658, 626...",4d2e06dbe1d7a07f3722947cb6e7f9e2487d4a536c73dc...,4d2e06dbe1d7a07f3722947cb6e7f9e2487d4a536c73dc...,6
1,mars.pdf,1,0,11,10359780639229817778,pdf,28aa3d03f33e8554459437f647fafca657b5360d0f7332...,5220,2024-12-03T23:41:33.764282,1.671919,mars.pdf,1554b6dd-8b79-4d75-9005-d39dfdb1439e,For more details about the Solar system see Ch...,#/texts/3,1,"[133.21472168, 570.30102539, 374.99798584, 584...",39a860697c7d1af540aa3134b499d476e713da79691077...,39a860697c7d1af540aa3134b499d476e713da79691077...,7
2,mars.pdf,1,0,11,10359780639229817778,pdf,28aa3d03f33e8554459437f647fafca657b5360d0f7332...,5220,2024-12-03T23:41:33.764282,1.671919,mars.pdf,1554b6dd-8b79-4d75-9005-d39dfdb1439e,"Mars, the fourth planet from the Sun, is a col...",#/texts/5,1,"[132.89646912, 500.11300659, 477.48300171, 538...",d3ea11dc437075f40b7de47fd2b8a1d334cbb13a20eb22...,d3ea11dc437075f40b7de47fd2b8a1d334cbb13a20eb22...,8
3,mars.pdf,1,0,11,10359780639229817778,pdf,28aa3d03f33e8554459437f647fafca657b5360d0f7332...,5220,2024-12-03T23:41:33.764282,1.671919,mars.pdf,1554b6dd-8b79-4d75-9005-d39dfdb1439e,· Distance from the Sun: Average of 228 millio...,#/texts/7,1,"[145.94500732, 452.29199219, 477.48199463, 478...",78223c017bd6ad6c4bd9dd9b5b5d5974c875c559a0be9f...,78223c017bd6ad6c4bd9dd9b5b5d5974c875c559a0be9f...,9
4,mars.pdf,1,0,11,10359780639229817778,pdf,28aa3d03f33e8554459437f647fafca657b5360d0f7332...,5220,2024-12-03T23:41:33.764282,1.671919,mars.pdf,1554b6dd-8b79-4d75-9005-d39dfdb1439e,1,#/texts/10,1,"[303.02359009, 86.47599792, 308.11401367, 100....",6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,10
5,earth.pdf,1,0,11,17915699055171962696,pdf,04e935746bc6dfb41912dd108bb10dde21c60b0da3f8fa...,4992,2024-12-03T23:41:32.088316,1.833233,earth.pdf,7cb164c0-cadf-4b65-a525-3c2d1a553128,Our solar system is a vast and fascinating exp...,#/texts/2,1,"[132.6648407, 588.23297119, 479.40899658, 626....",4d2e06dbe1d7a07f3722947cb6e7f9e2487d4a536c73dc...,4d2e06dbe1d7a07f3722947cb6e7f9e2487d4a536c73dc...,0
6,earth.pdf,1,0,11,17915699055171962696,pdf,04e935746bc6dfb41912dd108bb10dde21c60b0da3f8fa...,4992,2024-12-03T23:41:32.088316,1.833233,earth.pdf,7cb164c0-cadf-4b65-a525-3c2d1a553128,For more details about our Solar system see Ch...,#/texts/3,1,"[133.18649292, 570.30102539, 375.57901001, 584...",6b1799aadb8131a788033660ec73788c9f4165eda5e339...,6b1799aadb8131a788033660ec73788c9f4165eda5e339...,1
7,earth.pdf,1,0,11,17915699055171962696,pdf,04e935746bc6dfb41912dd108bb10dde21c60b0da3f8fa...,4992,2024-12-03T23:41:32.088316,1.833233,earth.pdf,7cb164c0-cadf-4b65-a525-3c2d1a553128,Earth is the third planet from the Sun. It's o...,#/texts/5,1,"[133.06599426, 512.06799316, 477.86444092, 538...",0408c7d1d9f634bdcc5d56eb76c4ab4578d8347fe94d9b...,0408c7d1d9f634bdcc5d56eb76c4ab4578d8347fe94d9b...,2
8,earth.pdf,1,0,11,17915699055171962696,pdf,04e935746bc6dfb41912dd108bb10dde21c60b0da3f8fa...,4992,2024-12-03T23:41:32.088316,1.833233,earth.pdf,7cb164c0-cadf-4b65-a525-3c2d1a553128,Basic facts about Earth:,#/texts/6,1,"[133.3850708, 494.13500977, 240.17199707, 508....",117c26c32f0a3ba6bfb7ccafab09213e415296ae97150d...,117c26c32f0a3ba6bfb7ccafab09213e415296ae97150d...,3
9,earth.pdf,1,0,11,17915699055171962696,pdf,04e935746bc6dfb41912dd108bb10dde21c60b0da3f8fa...,4992,2024-12-03T23:41:32.088316,1.833233,earth.pdf,7cb164c0-cadf-4b65-a525-3c2d1a553128,· Distance from the Sun: Average of 149.6 mill...,#/texts/7,1,"[145.94500732, 464.24700928, 477.480010

## Step-6: Exact Dedup



### 6.1 - Set Input/output Folder

In [21]:
STAGE  = 4

input_folder = output_docid_dir # previous output folder is the input folder for the current stage
output_folder =  output_exact_dedupe_dir

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-4: Processing input='output/03_docid_out' --> output='output/04_exact_dedupe_out'


### 6.2 - Execute

In [22]:
%%time

from data_processing_ray.runtime.ray import RayTransformLauncher
from ededup_transform_ray import EdedupRayTransformRuntimeConfiguration


# Prepare the commandline params
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # ededup parameters
    "ededup_hash_cpu": 0.5,
    "ededup_num_hashes": 2,
    "ededup_doc_column": "contents",
    "ededup_doc_id_column": "chunk_hash",
}

# Pass the commandline params
sys.argv = ParamsUtils.dict_to_req(d=params)

# create launcher
launcher = RayTransformLauncher(EdedupRayTransformRuntimeConfiguration())
# launch
return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

23:42:18 INFO - exact dedup params are {'doc_column': 'contents', 'doc_id_column': 'chunk_hash', 'use_snapshot': False, 'snapshot_directory': None, 'hash_cpu': 0.5, 'num_hashes': 2}
23:42:18 INFO - pipeline id pipeline_id
23:42:18 INFO - code location None
23:42:18 INFO - number of workers 2 worker options {'num_cpus': 1, 'max_restarts': -1}
23:42:18 INFO - actor creation delay 0
23:42:18 INFO - job details {'job category': 'preprocessing', 'job name': 'ededup', 'job type': 'ray', 'job id': 'job_id'}
23:42:18 INFO - data factory data_ is using local data access: input_folder - output/03_docid_out output_folder - output/04_exact_dedupe_out
23:42:18 INFO - data factory data_ max_files -1, n_sample -1
23:42:18 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
23:42:18 INFO - Running locally
2024-12-03 23:42:18,985	INFO worker.py:1777 -- Started a local Ray instance. View the dash

✅ Stage:4 completed successfully
CPU times: user 132 ms, sys: 143 ms, total: 275 ms
Wall time: 14.7 s


### 6.3 - Inspect Generated output

In [23]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)
print (f"Input chunks before exact dedupe : {input_df.shape[0]:,}")
print (f"Output chunks after exact dedupe : {output_df.shape[0]:,}")
print ("Duplicate chunks removed :  ", (input_df.shape[0] - output_df.shape[0]))

output_df.head(10)

Input data dimensions (rows x columns)=  (11, 19)
Output data dimensions (rows x columns)=  (9, 20)
Input chunks before exact dedupe : 11
Output chunks after exact dedupe : 9
Duplicate chunks removed :   2


,filename,num_pages,num_tables,num_doc_elements,document_hash,ext,hash,size,date_acquired,pdf_convert_time,source_filename,source_document_id,contents,doc_jsonpath,page_number,bbox,document_id,chunk_hash,chunk_id,removed
0,mars.pdf,1,0,11,10359780639229817778,pdf,28aa3d03f33e8554459437f647fafca657b5360d0f7332...,5220,2024-12-03T23:41:33.764282,1.671919,mars.pdf,1554b6dd-8b79-4d75-9005-d39dfdb1439e,Our solar system is a vast and fascinating exp...,#/texts/2,1,"[132.68968201, 588.23297119, 479.40899658, 626...",4d2e06dbe1d7a07f3722947cb6e7f9e2487d4a536c73dc...,4d2e06dbe1d7a07f3722947cb6e7f9e2487d4a536c73dc...,6,[]
1,mars.pdf,1,0,11,10359780639229817778,pdf,28aa3d03f33e8554459437f647fafca657b5360d0f7332...,5220,2024-12-03T23:41:33.764282,1.671919,mars.pdf,1554b6dd-8b79-4d75-9005-d39dfdb1439e,For more details about the Solar system see Ch...,#/texts/3,1,"[133.21472168, 570.30102539, 374.99798584, 584...",39a860697c7d1af540aa3134b499d476e713da79691077...,39a860697c7d1af540aa3134b499d476e713da79691077...,7,[]
2,mars.pdf,1,0,11,10359780639229817778,pdf,28aa3d03f33e8554459437f647fafca657b5360d0f7332...,5220,2024-12-03T23:41:33.764282,1.671919,mars.pdf,1554b6dd-8b79-4d75-9005-d39dfdb1439e,"Mars, the fourth planet from the Sun, is a col...",#/texts/5,1,"[132.89646912, 500.11300659, 477.48300171, 538...",d3ea11dc437075f40b7de47fd2b8a1d334cbb13a20eb22...,d3ea11dc437075f40b7de47fd2b8a1d334cbb13a20eb22...,8,[]
3,mars.pdf,1,0,11,10359780639229817778,pdf,28aa3d03f33e8554459437f647fafca657b5360d0f7332...,5220,2024-12-03T23:41:33.764282,1.671919,mars.pdf,1554b6dd-8b79-4d75-9005-d39dfdb1439e,· Distance from the Sun: Average of 228 millio...,#/texts/7,1,"[145.94500732, 452.29199219, 477.48199463, 478...",78223c017bd6ad6c4bd9dd9b5b5d5974c875c559a0be9f...,78223c017bd6ad6c4bd9dd9b5b5d5974c875c559a0be9f...,9,[]
4,mars.pdf,1,0,11,10359780639229817778,pdf,28aa3d03f33e8554459437f647fafca657b5360d0f7332...,5220,2024-12-03T23:41:33.764282,1.671919,mars.pdf,1554b6dd-8b79-4d75-9005-d39dfdb1439e,1,#/texts/10,1,"[303.02359009, 86.47599792, 308.11401367, 100....",6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,10,[]
5,earth.pdf,1,0,11,17915699055171962696,pdf,04e935746bc6dfb41912dd108bb10dde21c60b0da3f8fa...,4992,2024-12-03T23:41:32.088316,1.833233,earth.pdf,7cb164c0-cadf-4b65-a525-3c2d1a553128,For more details about our Solar system see Ch...,#/texts/3,1,"[133.18649292, 570.30102539, 375.57901001, 584...",6b1799aadb8131a788033660ec73788c9f4165eda5e339...,6b1799aadb8131a788033660ec73788c9f4165eda5e339...,1,[4d2e06dbe1d7a07f3722947cb6e7f9e2487d4a536c73d...
6,earth.pdf,1,0,11,17915699055171962696,pdf,04e935746bc6dfb41912dd108bb10dde21c60b0da3f8fa...,4992,2024-12-03T23:41:32.088316,1.833233,earth.pdf,7cb164c0-cadf-4b65-a525-3c2d1a553128,Earth is the third planet from the Sun. It's o...,#/texts/5,1,"[133.06599426, 512.06799316, 477.86444092, 538...",0408c7d1d9f634bdcc5d56eb76c4ab4578d8347fe94d9b...,0408c7d1d9f634bdcc5d56eb76c4ab4578d8347fe94d9b...,2,[]
7,earth.pdf,1,0,11,17915699055171962696,pdf,04e935746bc6dfb41912dd108bb10dde21c60b0da3f8fa...,4992,2024-12-03T23:41:32.088316,1.833233,earth.pdf,7cb164c0-cadf-4b65-a525-3c2d1a553128,Basic facts about Earth:,#/texts/6,1,"[133.3850708, 494.13500977, 240.17199707, 508....",117c26c32f0a3ba6bfb7ccafab09213e415296ae97150d...,117c26c32f0a3ba6bfb7ccafab09213e415296ae97150d...,3,[]
8,earth.pdf,1,0,11,17915699055171962696,pdf,04e935746bc6dfb41912dd108bb10dde21c60b0da3f8fa...,4992,2024-12-03T23:41:32.088316,1.833233,earth.pdf,7cb164c0-cadf-4b65-a525-3c2d1a553128,· Distance from the Sun: Average of 149.6 mill...,#/texts/7,1,"[145.94500732, 464.24700928, 477.48001099, 490...",83956bf14ac5123da4cb32147974bc6d13b0a8e9368f47...,83956bf14ac5123da4cb32147974bc6d13b0a8e9368f47...,4,[]


In [24]:
output_df[['filename', 'contents']]

,filename,contents
0,mars.pdf,Our solar system is a vast and fascinating exp...
1,mars.pdf,For more details about the Solar system see Ch...
2,mars.pdf,"Mars, the fourth planet from the Sun, is a col..."
3,mars.pdf,· Distance from the Sun: Average of 228 millio...
4,mars.pdf,1
5,earth.pdf,For more details about our Solar system see Ch...
6,earth.pdf,Earth is the third planet from the Sun. It's o...
7,earth.pdf,Basic facts about Earth:
8,earth.pdf,· Distance from the Sun: Average of 149.6 mill...


In [25]:
for f in output_df['filename'].unique():
    print ('==========' , f, '===========')
    chunks = output_df[output_df['filename'] == f]['contents']
    for idx , chunk in enumerate(chunks):
        print (f'-------Chunk {idx}------\n{chunk}\n-------')

========== mars.pdf ===========
-------Chunk 0------
Our solar system is a vast and fascinating expanse, comprising eight planets, ﬁve dwarf planets, numerous moons, asteroids, comets, and other celestial bodies. At its center lies the star we call the Sun.
-------
-------Chunk 1------
For more details about the Solar system see Chapter 1.
-------
-------Chunk 2------
Mars, the fourth planet from the Sun, is a cold, desert world with a thin atmosphere composed primarily of carbon dioxide. Its reddish hue comes from iron oxide, or rust, prevalent on its surface.
-------
-------Chunk 3------
· Distance from the Sun: Average of 228 million kilometers (142 million miles)
· Moons: Two small moons, Phobos and Deimos.
· Rotation Period: 24.6 hours (one Martian day - called a "sol")
-------
-------Chunk 4------
1
-------
========== earth.pdf ===========
-------Chunk 0------
For more details about our Solar system see Chapter 1.
-------
-------Chunk 1------
Earth is the third planet from the Su

### 6.4 - Understanding the output

Remember we had 8 chunks initially.  Now we have 7!  One duplicate chunk is removed.

If you look at the PDF, the following common paragraph in `earth.pdf` and `mars.pdf`  is removed from one of the documents!  Pretty neat, eh!

```text
## Solar System

Our solar system is a vast and fascinating expanse, comprising eight planets, five dwarf planets, numerous moons, asteroids, comets, and other celestial bodies. At its center lies the star we call the Sun.
```

## Step-7: Fuzzy Dedup

Post exact deduplication, fuzzy deduplication is applied with the goal of removing code files that may have **slight variations** and thereby unbiasing
the data further.

Small variations are quite commonly seen in code data in the form of variations in the values of variables, addittion of logging statements etc.

### 7.1 - Set Input/output Folder

In [26]:
## Input to this component is the output of doc_id generator component.

STAGE  = 5

input_folder = output_docid_dir # previous output folder is the input folder for the current stage
output_folder =  output_fuzzy_dedupe_dir

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-5: Processing input='output/03_docid_out' --> output='output/05_fuzzy_dedupe_out'


### 7.2 - Execute

In [27]:
%%time

import os
import sys

from data_processing.utils import ParamsUtils
from fdedup_transform_ray import FdedupRayTransformConfiguration
from data_processing_ray.runtime.ray import RayTransformLauncher

# create parameters

local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
code_location = {"github": "github", "commit_hash": "12345", "path": "path"}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # Orchestration parameters
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # columns used
    "fdedup_doc_column": "contents",
    "fdedup_id_column": "chunk_id",
    "fdedup_cluster_column": "chunk_hash",
    # infrastructure
    "fdedup_bucket_cpu": 0.3,
    "fdedup_doc_cpu": 0.3,
    "fdedup_mhash_cpu": 0.3,
    "fdedup_num_doc_actors": 1,
    "fdedup_num_bucket_actors": 1,
    "fdedup_num_minhash_actors": 1,
    "fdedup_num_preprocessors": 1,
    # fuzzy parameters
    "fdedup_num_permutations": 64,
    "fdedup_threshold": 0.7, # (default 0.8)
    "fdedup_shingles_size": 5,
    "fdedup_delimiters": " "
}

# Pass commandline params
sys.argv = ParamsUtils.dict_to_req(d=params)

# launch

launcher = RayTransformLauncher(FdedupRayTransformConfiguration())

return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

23:42:32 INFO - fuzzy dedup params are {'doc_column': 'contents', 'id_column': 'chunk_id', 'cluster_column': 'chunk_hash', 'bucket_cpu': 0.3, 'mhash_cpu': 0.3, 'doc_cpu': 0.3, 'num_doc_actors': 1, 'num_minhash_actors': 1, 'num_bucket_actors': 1, 'num_preprocessors': 1, 'num_permutations': 64, 'threshold': 0.7, 'shingles_size': 5, 'delimiters': ' ', 'snapshot_delay': 1, 'use_bucket_snapshot': False, 'use_doc_snapshot': False, 'random_delay_limit': 10, 'worker_options': {'num_cpus': 1}}
23:42:32 INFO - pipeline id pipeline_id
23:42:32 INFO - code location None
23:42:32 INFO - number of workers 2 worker options {'num_cpus': 1, 'max_restarts': -1}
23:42:32 INFO - actor creation delay 0
23:42:32 INFO - job details {'job category': 'preprocessing', 'job name': 'fdedup', 'job type': 'ray', 'job id': 'job_id'}
23:42:32 INFO - data factory data_ is using local data access: input_folder - output/03_docid_out output_folder - output/05_fuzzy_dedupe_out
23:42:32 INFO - data factory data_ max_files 

✅ Stage:5 completed successfully
CPU times: user 194 ms, sys: 182 ms, total: 376 ms
Wall time: 32 s


### 7.3 - Inspect Generated output

In [28]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)
print ("Duplicate chunks removed  by fuzzy-dedupe:  ", (input_df.shape[0] - output_df.shape[0]))

output_df.head(10)

Input data dimensions (rows x columns)=  (11, 19)
Output data dimensions (rows x columns)=  (8, 19)
Duplicate chunks removed  by fuzzy-dedupe:   3


,filename,num_pages,num_tables,num_doc_elements,document_hash,ext,hash,size,date_acquired,pdf_convert_time,source_filename,source_document_id,contents,doc_jsonpath,page_number,bbox,document_id,chunk_id,chunk_hash
0,mars.pdf,1,0,11,10359780639229817778,pdf,28aa3d03f33e8554459437f647fafca657b5360d0f7332...,5220,2024-12-03T23:41:33.764282,1.671919,mars.pdf,1554b6dd-8b79-4d75-9005-d39dfdb1439e,Our solar system is a vast and fascinating exp...,#/texts/2,1,"[132.68968201, 588.23297119, 479.40899658, 626...",4d2e06dbe1d7a07f3722947cb6e7f9e2487d4a536c73dc...,6,-1
1,mars.pdf,1,0,11,10359780639229817778,pdf,28aa3d03f33e8554459437f647fafca657b5360d0f7332...,5220,2024-12-03T23:41:33.764282,1.671919,mars.pdf,1554b6dd-8b79-4d75-9005-d39dfdb1439e,"Mars, the fourth planet from the Sun, is a col...",#/texts/5,1,"[132.89646912, 500.11300659, 477.48300171, 538...",d3ea11dc437075f40b7de47fd2b8a1d334cbb13a20eb22...,8,-1
2,mars.pdf,1,0,11,10359780639229817778,pdf,28aa3d03f33e8554459437f647fafca657b5360d0f7332...,5220,2024-12-03T23:41:33.764282,1.671919,mars.pdf,1554b6dd-8b79-4d75-9005-d39dfdb1439e,· Distance from the Sun: Average of 228 millio...,#/texts/7,1,"[145.94500732, 452.29199219, 477.48199463, 478...",78223c017bd6ad6c4bd9dd9b5b5d5974c875c559a0be9f...,9,-1
3,mars.pdf,1,0,11,10359780639229817778,pdf,28aa3d03f33e8554459437f647fafca657b5360d0f7332...,5220,2024-12-03T23:41:33.764282,1.671919,mars.pdf,1554b6dd-8b79-4d75-9005-d39dfdb1439e,1,#/texts/10,1,"[303.02359009, 86.47599792, 308.11401367, 100....",6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,10,5
4,earth.pdf,1,0,11,17915699055171962696,pdf,04e935746bc6dfb41912dd108bb10dde21c60b0da3f8fa...,4992,2024-12-03T23:41:32.088316,1.833233,earth.pdf,7cb164c0-cadf-4b65-a525-3c2d1a553128,For more details about our Solar system see Ch...,#/texts/3,1,"[133.18649292, 570.30102539, 375.57901001, 584...",6b1799aadb8131a788033660ec73788c9f4165eda5e339...,1,7
5,earth.pdf,1,0,11,17915699055171962696,pdf,04e935746bc6dfb41912dd108bb10dde21c60b0da3f8fa...,4992,2024-12-03T23:41:32.088316,1.833233,earth.pdf,7cb164c0-cadf-4b65-a525-3c2d1a553128,Earth is the third planet from the Sun. It's o...,#/texts/5,1,"[133.06599426, 512.06799316, 477.86444092, 538...",0408c7d1d9f634bdcc5d56eb76c4ab4578d8347fe94d9b...,2,-1
6,earth.pdf,1,0,11,17915699055171962696,pdf,04e935746bc6dfb41912dd108bb10dde21c60b0da3f8fa...,4992,2024-12-03T23:41:32.088316,1.833233,earth.pdf,7cb164c0-cadf-4b65-a525-3c2d1a553128,Basic facts about Earth:,#/texts/6,1,"[133.3850708, 494.13500977, 240.17199707, 508....",117c26c32f0a3ba6bfb7ccafab09213e415296ae97150d...,3,-1
7,earth.pdf,1,0,11,17915699055171962696,pdf,04e935746bc6dfb41912dd108bb10dde21c60b0da3f8fa...,4992,2024-12-03T23:41:32.088316,1.833233,earth.pdf,7cb164c0-cadf-4b65-a525-3c2d1a553128,· Distance from the Sun: Average of 149.6 mill...,#/texts/7,1,"[145.94500732, 464.24700928, 477.48001099, 490...",83956bf14ac5123da4cb32147974bc6d13b0a8e9368f47...,4,-1


In [29]:
output_df[['filename', 'contents']]

,filename,contents
0,mars.pdf,Our solar system is a vast and fascinating exp...
1,mars.pdf,"Mars, the fourth planet from the Sun, is a col..."
2,mars.pdf,· Distance from the Sun: Average of 228 millio...
3,mars.pdf,1
4,earth.pdf,For more details about our Solar system see Ch...
5,earth.pdf,Earth is the third planet from the Sun. It's o...
6,earth.pdf,Basic facts about Earth:
7,earth.pdf,· Distance from the Sun: Average of 149.6 mill...


In [30]:
for f in output_df['filename'].unique():
    print ('==========' , f, '===========')
    chunks = output_df[output_df['filename'] == f]['contents']
    for idx , chunk in enumerate(chunks):
        print (f'-------Chunk {idx}------\n{chunk}\n-------')

========== mars.pdf ===========
-------Chunk 0------
Our solar system is a vast and fascinating expanse, comprising eight planets, ﬁve dwarf planets, numerous moons, asteroids, comets, and other celestial bodies. At its center lies the star we call the Sun.
-------
-------Chunk 1------
Mars, the fourth planet from the Sun, is a cold, desert world with a thin atmosphere composed primarily of carbon dioxide. Its reddish hue comes from iron oxide, or rust, prevalent on its surface.
-------
-------Chunk 2------
· Distance from the Sun: Average of 228 million kilometers (142 million miles)
· Moons: Two small moons, Phobos and Deimos.
· Rotation Period: 24.6 hours (one Martian day - called a "sol")
-------
-------Chunk 3------
1
-------
========== earth.pdf ===========
-------Chunk 0------
For more details about our Solar system see Chapter 1.
-------
-------Chunk 1------
Earth is the third planet from the Sun. It's our home planet. Earth is the only place we know of with life.
-------
-----

### 7.4- Understanding the output

So we started with 7 rows and ended up with 6.  Fuzzy dedupe removed the following **very similar** chunk.

These are pretty similar chunks except for the words 'the' and 'our'

**earth.pdf**

`For more details about *our* Solar system see Chapter 1.`

**mars.pdf**

`For more details about *the* Solar system see Chapter 1.`

Pretty neat, eh? 👏

### Configuring Fuzzy de-dupe

You can tweak fuzzy dedupe by tweaking the following parameters

```python
# fuzzy parameters
    "fdedup_num_permutations": 64,
    "fdedup_threshold": 0.7, #  (default 0.8)
    "fdedup_shingles_size": 5,
    "fdedup_delimiters": " "
```

In our case, we set `fdedup_threshold` parameter to 0.7.  


## Step-8:   Text encoding

Encode text for the vector storage.

### 8.1 - Set Input/output Folder

In [31]:
STAGE  = 6

input_folder = output_fuzzy_dedupe_dir # previous output folder is the input folder for the current stage
output_folder =  output_embeddings_dir

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-6: Processing input='output/05_fuzzy_dedupe_out' --> output='output/06_embeddings_out'


### 8.2 - Execute

In [32]:
%%time

from text_encoder_transform_ray import TextEncoderRayTransformConfiguration

local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # text_encoder
    "text_encoder_model_name": MY_CONFIG.EMBEDDING_MODEL,
}

sys.argv = ParamsUtils.dict_to_req(d=params)
# create launcher
launcher = RayTransformLauncher(TextEncoderRayTransformConfiguration())
# Launch the ray actor(s) to process the input

return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

23:43:05 INFO - text_encoder parameters are : {'content_column_name': 'contents', 'output_embeddings_column_name': 'embeddings', 'model_name': 'sentence-transformers/all-MiniLM-L6-v2'}
23:43:05 INFO - pipeline id pipeline_id
23:43:05 INFO - code location None
23:43:05 INFO - number of workers 2 worker options {'num_cpus': 1, 'max_restarts': -1}
23:43:05 INFO - actor creation delay 0
23:43:05 INFO - job details {'job category': 'preprocessing', 'job name': 'text_encoder', 'job type': 'ray', 'job id': 'job_id'}
23:43:05 INFO - data factory data_ is using local data access: input_folder - output/05_fuzzy_dedupe_out output_folder - output/06_embeddings_out
23:43:05 INFO - data factory data_ max_files -1, n_sample -1
23:43:05 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
23:43:05 INFO - Running locally
2024-12-03 23:43:06,229	INFO worker.py:1777 -- Started a local Ray instance.

✅ Stage:6 completed successfully
CPU times: user 438 ms, sys: 164 ms, total: 602 ms
Wall time: 21.9 s


### 8.3 - Inspect Generated output

You will see a column called `embeddings` added at the end.  This the text content converted into vectors or embeddings.  We used the model `sentence-transformers/all-MiniLM-L6-v2`

In [33]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.head(5)

Input data dimensions (rows x columns)=  (8, 19)
Output data dimensions (rows x columns)=  (8, 20)


,filename,num_pages,num_tables,num_doc_elements,document_hash,ext,hash,size,date_acquired,pdf_convert_time,source_filename,source_document_id,contents,doc_jsonpath,page_number,bbox,document_id,chunk_id,chunk_hash,embeddings
0,mars.pdf,1,0,11,10359780639229817778,pdf,28aa3d03f33e8554459437f647fafca657b5360d0f7332...,5220,2024-12-03T23:41:33.764282,1.671919,mars.pdf,1554b6dd-8b79-4d75-9005-d39dfdb1439e,Our solar system is a vast and fascinating exp...,#/texts/2,1,"[132.68968201, 588.23297119, 479.40899658, 626...",4d2e06dbe1d7a07f3722947cb6e7f9e2487d4a536c73dc...,6,-1,"[0.0021121458, -0.010703966, 0.011550098, 0.02..."
1,mars.pdf,1,0,11,10359780639229817778,pdf,28aa3d03f33e8554459437f647fafca657b5360d0f7332...,5220,2024-12-03T23:41:33.764282,1.671919,mars.pdf,1554b6dd-8b79-4d75-9005-d39dfdb1439e,"Mars, the fourth planet from the Sun, is a col...",#/texts/5,1,"[132.89646912, 500.11300659, 477.48300171, 538...",d3ea11dc437075f40b7de47fd2b8a1d334cbb13a20eb22...,8,-1,"[0.08522691, 0.019003406, -0.03933215, 0.05468..."
2,mars.pdf,1,0,11,10359780639229817778,pdf,28aa3d03f33e8554459437f647fafca657b5360d0f7332...,5220,2024-12-03T23:41:33.764282,1.671919,mars.pdf,1554b6dd-8b79-4d75-9005-d39dfdb1439e,· Distance from the Sun: Average of 228 millio...,#/texts/7,1,"[145.94500732, 452.29199219, 477.48199463, 478...",78223c017bd6ad6c4bd9dd9b5b5d5974c875c559a0be9f...,9,-1,"[0.06868917, 0.014957822, 0.011387792, 0.05219..."
3,mars.pdf,1,0,11,10359780639229817778,pdf,28aa3d03f33e8554459437f647fafca657b5360d0f7332...,5220,2024-12-03T23:41:33.764282,1.671919,mars.pdf,1554b6dd-8b79-4d75-9005-d39dfdb1439e,1,#/texts/10,1,"[303.02359009, 86.47599792, 308.11401367, 100....",6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,10,5,"[-0.04356465, 0.0018598451, -0.051165756, 0.06..."
4,earth.pdf,1,0,11,17915699055171962696,pdf,04e935746bc6dfb41912dd108bb10dde21c60b0da3f8fa...,4992,2024-12-03T23:41:32.088316,1.833233,earth.pdf,7cb164c0-cadf-4b65-a525-3c2d1a553128,For more details about our Solar system see Ch...,#/texts/3,1,"[133.18649292, 570.30102539, 375.57901001, 584...",6b1799aadb8131a788033660ec73788c9f4165eda5e339...,1,7,"[-0.060079515, 0.01474668, 0.025538314, 0.0483..."


## Step-9: Copy output to final output dir

In [34]:
import shutil

shutil.rmtree(MY_CONFIG.OUTPUT_FOLDER_FINAL, ignore_errors=True)
shutil.copytree(src=output_folder, dst=MY_CONFIG.OUTPUT_FOLDER_FINAL)

print (f"✅ Copied output from '{output_folder}' --> '{MY_CONFIG.OUTPUT_FOLDER_FINAL}'")

✅ Copied output from 'output/06_embeddings_out' --> 'output/output_final'
